In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import statsmodels.api as sm

## Read in 2003 Data

In [2]:
#Read in files 2003 Consumption
consump_03_01 = pd.read_csv('2003_Consumption/FILE01.csv')
consump_03_01.columns = [col.strip() for col in consump_03_01.columns]
consump_03_01 = consump_03_01[['PUBID8', 'REGION8', 'CENDIV8', 'SQFT8', 'PBA8', 'YRCON8', 'FREESTN8'
                              ,'GLSSPC8', 'NELVTR8', 'NESLTR8', 'OPEN248', 'WKHRS8',
                              'NWKER8']]

consump_03_02 = pd.read_csv('2003_Consumption/FILE02.csv')
consump_03_02.columns = [col.strip() for col in consump_03_02.columns]
consump_03_02 = consump_03_02[['PUBID8', 'ONEACT8']]

consump_03_04 = pd.read_csv('2003_Consumption/FILE04.csv')
consump_03_04.columns = [col.strip() for col in consump_03_04.columns]
consump_03_04 = consump_03_04[['PUBID8','RFGEQP8']]

consump_03_05 = pd.read_csv('2003_Consumption/FILE05.csv')
consump_03_05.columns = [col.strip() for col in consump_03_05.columns]
consump_03_05 = consump_03_05[['PUBID8','ELHT18', 'ELCOOL8','ELWATR8','ELCOOK8','ELMANU8']]

consump_03_15 = pd.read_csv('2003_Consumption/FILE15.csv')
consump_03_15.columns = [col.strip() for col in consump_03_15.columns]
consump_03_15 = consump_03_15[['PUBID8','ELCNS8']]

#Merge Dataframes
dfs = [consump_03_01, consump_03_02, consump_03_04, consump_03_05, consump_03_15]
consump_03 = reduce(lambda left,right: pd.merge(left,right,on='PUBID8'), dfs)

consump_03['YEAR'] = 2003

## Read in 2012 Data

In [3]:
consump_12 = pd.read_csv('2012_public_use_data_aug2016.csv')

consump_12 = consump_12[['PUBID', 'REGION', 'CENDIV', 'SQFT', 'PBA','YRCON','FREESTN', 'NELVTR','NESLTR', 'GLSSPC',
          'OPEN24', 'WKHRS', 'NWKER', 'ONEACT', 'RFGEQP', 'ELHT1', 'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU',
          'ELCNS']]

consump_12['YEAR'] = 2012

There are climate fields PUBCLIM in 2012 and CLIMAT in 2003 that refer to a climate type a building is located.  These are based on the number of heating and cooling days.  We could potentially use this as a consumption feature by tying zipcodes to NOAA data which has the heating/cooling days for each of the stations.

GlassPercent Categories are different, might need to standardize these if we actually use them

## Merge Years & Data Standarization 

In [4]:
for i in consump_03.columns:
    consump_03[i] = pd.to_numeric(consump_03[i], errors = 'coerce')
    
consump_03.columns = list(consump_12.columns)

consump_all = pd.concat([consump_12,consump_03])

In [5]:
PBA_Dict = {
1:'Vacant',
2:'Office',
4:'Laboratory',
5:'Nonrefrigerated warehouse',
6:'Food sales',
7:'Public order and safety',
8:'Outpatient health care',
11:'Refrigerated warehouse',
12:'Religious worship',
13:'Public assembly',
14:'Education',
15:'Food service',
16:'Inpatient health care',
17:'Nursing',
18:'Lodging',
23:'Strip shopping mall',
24:'Enclosed mall',
25:'Retail other than mall',
26:'Service',
91: 'Other'}

In [6]:
pba_list = []
for i in consump_all['PBA']:
    pba_list.append(PBA_Dict[i])

consump_all['PBA_Detail'] = pd.Series(pba_list)

In [7]:
consump_all['NELVTR'] = consump_all['NELVTR'].fillna(value=0)
consump_all['NESLTR'] = consump_all['NESLTR'].fillna(value=0)

In [8]:
#Filter for freestanding building with a single primary activity
consump_filtered = consump_all.loc[(consump_all['FREESTN'] == 1) & (consump_all['ONEACT'] == 1)]
consump_filtered = consump_filtered.dropna(axis=0, how = 'any')

binary_fix = ['OPEN24','RFGEQP', 'ELHT1','ELCOOL','ELWATR', 'ELCOOK', 'ELMANU']
for column in binary_fix:
    consump_filtered[column] = consump_filtered[column].replace(to_replace = 2, value = 0)
    
max_val_fix = ['NELVTR', 'NESLTR']
for column in binary_fix:
    consump_filtered[column] = consump_filtered[column].replace(to_replace = 995, value = 51)

# Regression Modeling 

In [9]:
consump_filtered.columns

Index(['PUBID', 'REGION', 'CENDIV', 'SQFT', 'PBA', 'YRCON', 'FREESTN',
       'NELVTR', 'NESLTR', 'GLSSPC', 'OPEN24', 'WKHRS', 'NWKER', 'ONEACT',
       'RFGEQP', 'ELHT1', 'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU', 'ELCNS',
       'YEAR', 'PBA_Detail'],
      dtype='object')

In [31]:
X = consump_filtered[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU']]
y = consump_filtered['ELCNS']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
#predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                     811.4
Date:                Sun, 25 Mar 2018   Prob (F-statistic):               0.00
Time:                        14:32:43   Log-Likelihood:                -95869.
No. Observations:                5687   AIC:                         1.918e+05
Df Residuals:                    5674   BIC:                         1.919e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const      -5.148e+05   2.64e+05     -1.948      0.051     -1.03e+06  3245.350
SQFT          15.8374      0.409     38.751      0.000        15.036    16.639
WKHRS      -3373.4664   2910.141     -1.159      0.246     -9078.454  2331.521
NWKER       3588.6533    180.100     19.926      0.000      3235.589  3941.718
OPEN24      2.358e+06   3.65e+05      6.466      0.000      1.64e+06  3.07e+06
NELVTR      1.676e+04   1633.912     10.258      0.000      1.36e+04     2e+04
NESLTR      3959.4814   1848.770      2.142      0.032       335.185  7583.778
RFGEQP     -1.101e+05   2.05e+05     -0.538      0.590     -5.11e+05  2.91e+05
ELHT1       8.994e+04   1.55e+05      0.581      0.561     -2.14e+05  3.94e+05
ELCOOL      2.625e+05   2.29e+05      1.148      0.251     -1.86e+05  7.11e+05
ELWATR     -7.321e+04   1.46e+05     -0.502      0.616     -3.59e+05  2.13e+05
ELCOOK      2.831e+05   1.58e+05      1.795      0.073      -2.6e+04  5.92e+05
ELMANU      2.114e+05   3.58e+05      0.591      0.555      -4.9e+05  9.13e+05
==============================================================================
Omnibus:                    11491.237   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         60047251.659
Skew:                          16.289   Prob(JB):                         0.00
Kurtosis:                     505.342   Cond. No.                     1.70e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.7e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
consump_filtered.groupby('PBA_Detail').sum()

PUBID  REGION  CENDIV       SQFT   PBA    YRCON  \
PBA_Detail                                                                     
Education                  2418818    1901    3747   61876609  9864  1326281   
Enclosed mall                98057      83     160   26487000   744    61353   
Food sales                  322581     309     614    2936501   668   219478   
Food service                995125     831    1644    5765652  4484   554339   
Inpatient health care      1117991     874    1721  198001502  5498   676424   
Laboratory                  130022      89     176    6366001   132    63591   
Lodging                     903245     725    1436   46466354  4720   500196   
Nonrefrigerated warehouse  2065561    1728    3472   80234359  3046  1165133   
Nursing                     275628     212     405    7921250  1462   168408   
Office                     3885374    3118    6123  162285567  2402  2129572   
Other                       251539     186     369    6513100  6037   128048   
Outpatient health care      576804     467     927    9507501  1360   325568   
Public assembly            1080691     857    1663   42922554  4270   601156   
Public order and safety     296067     232     453    9842500   637   170903   
Refrigerated warehouse       79789      53     101    4078000   233    38741   
Religious worship          1128084     872    1687   11061753  3960   596354   
Retail other than mall      968835     789    1550   13186950  7292   540332   
Service                    1145720     902    1760    9351651  9122   660090   
Strip shopping mall        1025018     837    1620   26474803  6854   592631   
Vacant                      411312     298     592    6318900   105   188997   

                           FREESTN   NELVTR  NESLTR  GLSSPC   ...     \
PBA_Detail                                                    ...      
Education                    706.0    410.0    73.0  1844.0   ...      
Enclosed mall                 31.0    127.0   149.0    59.0   ...      
Food sales                   112.0      4.0     1.0   256.0   ...      
Food service                 299.0     22.0   101.0   761.0   ...      
Inpatient health care        345.0  10024.0    99.0  1081.0   ...      
Laboratory                    33.0     96.0     0.0   109.0   ...      
Lodging                      264.0   1752.0  1183.0   772.0   ...      
Nonrefrigerated warehouse    610.0    114.0   147.0   982.0   ...      
Nursing                       86.0    152.0     2.0   240.0   ...      
Office                      1139.0   5806.0  1372.0  3457.0   ...      
Other                         68.0     27.0    11.0   110.0   ...      
Outpatient health care       170.0    213.0     6.0   468.0   ...      
Public assembly              331.0   1602.0  6157.0   754.0   ...      
Public order and safety       91.0    156.0    12.0   196.0   ...      
Refrigerated warehouse        21.0      7.0     4.0    24.0   ...      
Religious worship            330.0     71.0    82.0   785.0   ...      
Retail other than mall       293.0     66.0    53.0   607.0   ...      
Service                      354.0     46.0    46.0   638.0   ...      
Strip shopping mall          299.0     23.0    53.0   905.0   ...      
Vacant                       105.0     87.0     0.0   263.0   ...      

                              NWKER  ONEACT  RFGEQP  ELHT1  ELCOOL  ELWATR  \
PBA_Detail                                                                   
Education                   47332.0   706.0   601.0  187.0   619.0   242.0   
Enclosed mall               17216.0    31.0    31.0   13.0    31.0    24.0   
Food sales                   9081.0   112.0   111.0   54.0   105.0    72.0   
Food service                10112.0   299.0   299.0  108.0   277.0   117.0   
Inpatient health care      388501.0   345.0   344.0   24.0   318.0    59.0   
Laboratory                  10966.0    33.0    33.0    2.0    30.0     8.0   
Lodging                     26871.0   264.0   251.0  148.0 

# Business Specific Models

In [33]:
#Office Model

office_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == 'Office']

X = office_df[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU']]
y = office_df['ELCNS']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
#predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.694
Method:                 Least Squares   F-statistic:                     215.9
Date:                Sun, 25 Mar 2018   Prob (F-statistic):          7.74e-282
Time:                        14:34:56   Log-Likelihood:                -18729.
No. Observations:                1139   AIC:                         3.748e+04
Df Residuals:                    1126   BIC:                         3.755e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const        -5.5e+05   7.14e+05     -0.770      0.441     -1.95e+06  8.51e+05
SQFT          11.4215      0.752     15.181      0.000         9.945    12.898
WKHRS      -2898.9060   7596.823     -0.382      0.703     -1.78e+04   1.2e+04
NWKER       1884.3929    284.168      6.631      0.000      1326.835  2441.951
OPEN24      1.612e+06   9.25e+05      1.743      0.082     -2.03e+05  3.43e+06
NELVTR      1.711e+04   2242.483      7.632      0.000      1.27e+04  2.15e+04
NESLTR     -1091.1562   3448.832     -0.316      0.752     -7858.017  5675.705
RFGEQP      1.839e+05   2.88e+05      0.639      0.523     -3.81e+05  7.49e+05
ELHT1       4.177e+05   2.25e+05      1.857      0.064     -2.37e+04  8.59e+05
ELCOOL      6.377e+05    5.6e+05      1.138      0.255     -4.61e+05  1.74e+06
ELWATR     -2.721e+05   2.24e+05     -1.214      0.225     -7.12e+05  1.68e+05
ELCOOK      2.366e+05   2.71e+05      0.874      0.382     -2.94e+05  7.68e+05
ELMANU     -2.579e+05    6.5e+05     -0.397      0.691     -1.53e+06  1.02e+06
==============================================================================
Omnibus:                     2100.386   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3534751.073
Skew:                          12.753   Prob(JB):                         0.00
Kurtosis:                     274.718   Cond. No.                     3.28e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.28e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
#Food Service Model

food_serve_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == 'Religious worship']

X = food_serve_df[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU']]
y = food_serve_df['ELCNS']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
#predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                     1555.
Date:                Sun, 25 Mar 2018   Prob (F-statistic):          1.83e-273
Time:                        14:35:49   Log-Likelihood:                -4723.9
No. Observations:                 330   AIC:                             9474.
Df Residuals:                     317   BIC:                             9523.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const      -7.406e+04   1.01e+05     -0.730      0.466     -2.74e+05  1.26e+05
SQFT          18.4704      0.444     41.623      0.000        17.597    19.343
WKHRS       -327.7710    961.084     -0.341      0.733     -2218.680  1563.138
NWKER      -7573.2344    259.994    -29.128      0.000     -8084.767 -7061.702
OPEN24     -7.474e+04   1.67e+05     -0.448      0.655     -4.03e+05  2.54e+05
NELVTR     -3.532e+05   3.79e+04     -9.308      0.000     -4.28e+05 -2.79e+05
NESLTR      1.216e+06   4.64e+04     26.227      0.000      1.13e+06  1.31e+06
RFGEQP     -7.476e+04   7.14e+04     -1.048      0.296     -2.15e+05  6.56e+04
ELHT1       3.456e+04    5.4e+04      0.640      0.523     -7.17e+04  1.41e+05
ELCOOL     -5.223e+04   8.68e+04     -0.602      0.548     -2.23e+05  1.19e+05
ELWATR      1.021e+05   4.96e+04      2.057      0.040      4449.165     2e+05
ELCOOK      3.408e+04   4.95e+04      0.688      0.492     -6.34e+04  1.32e+05
ELMANU      1.481e+05   2.94e+05      0.503      0.615     -4.31e+05  7.27e+05
==============================================================================
Omnibus:                      135.451   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5747.100
Skew:                          -0.921   Prob(JB):                         0.00
Kurtosis:                      23.361   Cond. No.                     1.74e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.74e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [81]:
#Nonrefrigerated warehouse Model

ware_nofridge_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == 'Nonrefrigerated warehouse']

X = ware_nofridge_df[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU']]
y = ware_nofridge_df['ELCNS']
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     181.6
Date:                Sun, 25 Mar 2018   Prob (F-statistic):          2.31e-190
Time:                        15:08:28   Log-Likelihood:                -9743.4
No. Observations:                 610   AIC:                         1.951e+04
Df Residuals:                     598   BIC:                         1.956e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
SQFT           3.7029      0.469      7.904      0.000         2.783     4.623
WKHRS       1070.2875   2583.374      0.414      0.679     -4003.301  6143.876
NWKER       8703.9733    837.298     10.395      0.000      7059.572  1.03e+04
OPEN24       5.89e+04   4.22e+05      0.139      0.889     -7.71e+05  8.89e+05
NELVTR     -2.892e+05   1.59e+05     -1.820      0.069     -6.01e+05  2.29e+04
NESLTR      2.324e+05   9.48e+04      2.452      0.014      4.63e+04  4.19e+05
RFGEQP       7.26e+04   2.18e+05      0.333      0.739     -3.55e+05  5.01e+05
ELHT1       2.824e+05   2.02e+05      1.400      0.162     -1.14e+05  6.79e+05
ELCOOL     -2.327e+05   2.43e+05     -0.956      0.340     -7.11e+05  2.45e+05
ELWATR     -2.629e+05   1.93e+05     -1.361      0.174     -6.42e+05  1.16e+05
ELCOOK     -1.803e+05   4.14e+05     -0.435      0.663     -9.94e+05  6.33e+05
ELMANU      5.447e+05   2.63e+05      2.071      0.039      2.82e+04  1.06e+06
==============================================================================
Omnibus:                      932.134   Durbin-Watson:                   2.076
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           456497.375
Skew:                           8.361   Prob(JB):                         0.00
Kurtosis:                     135.970   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [82]:
model.pvalues

SQFT      1.309928e-14
WKHRS     6.788042e-01
NWKER     2.221288e-23
OPEN24    8.891676e-01
NELVTR    6.924169e-02
NESLTR    1.448901e-02
RFGEQP    7.390858e-01
ELHT1     1.620616e-01
ELCOOL    3.395190e-01
ELWATR    1.739714e-01
ELCOOK    6.634807e-01
ELMANU    3.877111e-02
dtype: float64

## Production Models

In [85]:
final_dependent = ['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR',
                   'RFGEQP', 'ELHT1','ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU']

In [86]:
ols_coefficients = []
ols_pvals = []
business_vals = []

for i in PBA_Dict.keys():
    temp_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == PBA_Dict[i]]
    
    X = temp_df[final_dependent]
    y = temp_df['ELCNS']
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()    
    
    ols_coefficients.append(model.params) 
    ols_pvals.append(model.pvalues)
    business_vals.append(PBA_Dict[i])


In [87]:
business_df = pd.DataFrame(business_vals)
business_df.columns = ['business_type']

ols_coef_df = pd.DataFrame(ols_coefficients)
ols_coef_df = pd.merge(ols_coef_df,business_df, left_index = True, right_index = True)

ols_pvals_df = pd.DataFrame(ols_pvals)
ols_pvals_df = pd.merge(ols_pvals_df,business_df, left_index = True, right_index = True)